Source: https://www.eia.gov/

In [300]:
from dotenv import load_dotenv
import pandas as pd
import json
import numpy
import seaborn 
import matplotlib.pyplot as plt
import requests
import os
import plotly.graph_objects as go

In [301]:
load_dotenv()

API_KEY = os.getenv("API_KEY")

# State Energy Data System (SEDS) category ID
SEDS_ID = os.getenv("SEDS_ID") 

# Generic URLs for making API calls on the category and series sides
CAT_URL = os.getenv("CAT_URL")
SERIES_URL = os.getenv("SERIES_URL")

Create a list of all 50 states.

In [302]:
state_names=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia" "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

Get the major categories roster

In [303]:
# Start at the SEDS level.
response = requests.get(CAT_URL, params={"api_key":API_KEY, "category_id": SEDS_ID})
major_cat_ids = {}

for i in response.json()['category']['childcategories']:
    name = i['name']
    cat_id = i['category_id']
    major_cat_ids[name] = cat_id
print(major_cat_ids)

{'Consumption': 40204, 'Prices & Expenditures': 40909, 'Production': 40207, 'Population': 40367, 'GDP': 40827, 'Conversion Factors': 40867}


#### Consumption

##### Total all sectors

In [304]:
# Total All Sectors 
tas_major_cats = requests.get(CAT_URL, params={"api_key":API_KEY, "category_id":os.getenv("CONS_ID")})
print(tas_major_cats.json()['category']['childcategories'])

[{'category_id': 40236, 'name': 'Total All Sectors'}, {'category_id': 40208, 'name': 'Total End-Use Sectors'}, {'category_id': 40209, 'name': 'Residential Sector'}, {'category_id': 40210, 'name': 'Commercial Sector'}, {'category_id': 40211, 'name': 'Industrial Sector'}, {'category_id': 40212, 'name': 'Transportation Sector'}, {'category_id': 40213, 'name': 'Electric Power Sector'}]


In [305]:
# Get total all sectors state series IDs
tas = requests.get(CAT_URL, params={"api_key":API_KEY, "category_id":os.getenv("TAS_ID")})

# Match states with their data series IDs
state_series_tas_ids = {}
for i in tas.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_tas_ids[name.lstrip("Total energy consum").lstrip("ption, ")] = series_id

# print(state_series_tas_ids)

# # Make folders for total energy consumption of each state
# root_dir = "./project_datasets/consumption/total"
# for key,val in state_series_tas_ids.items():
#     path = os.path.join(root_dir,key)
#     if os.path.isdir(path):
#         pass
#     else:
#         os.mkdir(path)

##### All Petroleum Products

In [306]:
petrol = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40441})

state_series_petrol_ids = {}
for i in petrol.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_petrol_ids[name.lstrip("All petroleum products total consumption, excluding").lstrip("biofuels, ")] = series_id

##### Coal

In [307]:
coal = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40478})

state_series_coal_ids = {}
for i in coal.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_coal_ids[name.lstrip("Coal total consumption, ")] = series_id

##### Natural Gas including Supplemental Gaseous Fuels

In [308]:
natgas = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40452})

state_series_natgas_ids = {}
for i in natgas.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_natgas_ids[name.lstrip("Natural gas total consumption, ")] = series_id

##### Solar Energy

In [309]:
solar = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40382})

state_series_solar_ids = {}
for i in solar.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_solar_ids[name.lstrip("Solar energy total consum").lstrip("ption, ")] = series_id

##### Wind Energy

In [310]:
wind = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40324})

state_series_wind_ids = {}
for i in wind.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_wind_ids[name.lstrip("Wind energy total consum").lstrip("ption, ")] = series_id

##### Wood and Waste Energy

In [311]:
wood = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40335})

state_series_wood_ids = {}
for i in wood.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_wood_ids[name.lstrip("Wood and waste energy total consum").lstrip("ption, ")] = series_id

##### Fossil Fuels

In [312]:
fossils = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40469})

state_series_fossils_ids = {}
for i in fossils.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_fossils_ids[name.lstrip("Fossil fuels total consum").lstrip("ption, ")] = series_id

##### Renewable Energy

In [313]:
renewables = requests.get(CAT_URL, params={"api_key": API_KEY, "category_id": 40425})

state_series_renewables_ids = {}
for i in renewables.json()['category']['childseries']:
    name = i['name']
    series_id = i['series_id']
    state_series_renewables_ids[name.lstrip("Renewable energy total consumption, ")] = series_id

##### Wrap it all up in a dataframe and graph it

In [314]:
consumption = pd.DataFrame(columns=["State", "Year", "Total", "Petroleum", "Coal", "Natural Gas", "Solar", "Wind", "Wood", "Fossil Fuels", "Renewables"])

energy_types = ["Total", "Petroleum", "Coal", "Natural Gas", "Solar", "Wind", "Wood", "Fossil Fuels", "Renewables"]

all_ids = [
    state_series_tas_ids,
    state_series_petrol_ids,
    state_series_coal_ids,
    state_series_natgas_ids,
    state_series_solar_ids,
    state_series_wind_ids,
    state_series_wood_ids,
    state_series_fossils_ids,
    state_series_renewables_ids
    ]

new_data = {i : numpy.NAN for i in ["State","Year","Total","Petroleum","Coal","Natural Gas","Solar","Wind","Wood","Fossil Fuels","Renewables"]}

for i in range(len(energy_types)):
    type = energy_types[i]
    id_list = all_ids[i]
    print(f"Working on {type} energy")

    for key,val in id_list.items(): # Loop through all states
        state = key
        yearly_req = requests.get(SERIES_URL, params={"api_key":API_KEY, "series_id":val})
        yearly_data = yearly_req.json()['series'][-1]['data']  

        for i in yearly_data: # Get consumption by year for energy type
            year = i[0]
            data = i[1]
            new_data[type] = data
            new_data["State"] = state
            new_data["Year"] = year
            consumption = consumption.append(new_data,ignore_index=True)
            new_data = {}

# Clean up the dataframe by combining matching rows and eliminating NAN values.
consumption = consumption.groupby(["State","Year"])[["Total","Petroleum","Coal","Natural Gas","Solar","Wind","Wood","Fossil Fuels","Renewables"]].first().reset_index()

Working on Total energy
Working on Petroleum energy
Working on Coal energy
Working on Natural Gas energy
Working on Solar energy
Working on Wind energy
Working on Wood energy
Working on Fossil Fuels energy
Working on Renewables energy


In [315]:
state = "Washington"
filter = consumption["State"] == state
ex = consumption[filter]
fig = go.Figure()

colors = ['darkslateblue', 'darkslategray', 'darkturquoise', 'slateblue','lightskyblue','lightsteelblue','midnightblue','powderblue','mediumaquamarine']

for i in range(0,len(energy_types)):
    fig.add_trace(
        go.Scatter(x=list(ex.Year),
         y=list(ex[energy_types[i]]),
         name=energy_types[i],
         mode = "lines",
         line = dict(width=2,color=colors[i])
        )
    )

# Set title
fig.update_layout(
    title_text=f"Annual Energy Consumption, Billion Btus - {state}"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=10,
                     label="10Y",
                     step="year",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

#### Production

In [316]:
response = requests.get(CAT_URL, params={"api_key":API_KEY, "category_id":40207})
print(response.json())


{'request': {'category_id': 40207, 'command': 'category'}, 'category': {'category_id': '40207', 'parent_category_id': '40203', 'name': 'Production', 'notes': '', 'childcategories': [{'category_id': 40910, 'name': 'Coal'}, {'category_id': 40911, 'name': 'Crude Oil including Lease Condensate'}, {'category_id': 40912, 'name': 'Nuclear Power'}, {'category_id': 40913, 'name': 'Natural Gas Marketed'}, {'category_id': 40426, 'name': 'Renewable Energy'}], 'childseries': [{'series_id': 'SEDS.TEPRB.AK.A', 'name': 'Total primary energy production, Alaska', 'f': 'A', 'units': 'Billion Btu', 'updated': '24-JUN-21 07.42.03 PM'}, {'series_id': 'SEDS.TEPRB.AL.A', 'name': 'Total primary energy production, Alabama', 'f': 'A', 'units': 'Billion Btu', 'updated': '24-JUN-21 07.42.03 PM'}, {'series_id': 'SEDS.TEPRB.AR.A', 'name': 'Total primary energy production, Arkansas', 'f': 'A', 'units': 'Billion Btu', 'updated': '24-JUN-21 07.42.03 PM'}, {'series_id': 'SEDS.TEPRB.AZ.A', 'name': 'Total primary energy p

#### GDP

In [317]:
energy = {
    "Alaska": {
        "Total Produced":0,
        "Total Consumed":0
    },
    "Alabama": {
        "Total Produced":0,
        "Total Consumed":0
    }
}

In [318]:
import plotly.graph_objects as go

In [319]:
state = consumption["State"] == "United States"
ex = consumption[state]
fig = go.Figure()

for i in energy_types:
    fig.add_trace(
        go.Scatter(x=list(ex.Year),
         y=list(ex[i]),
         name=i,
        )
    )

# fig.add_trace(
#     go.Scatter(x=list(ex.Year), y=list(ex.Petroleum)))

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=10,
                     label="10Y",
                     step="year",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()
